In [12]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("yolov9_paper.pdf")
data = loader.load()

In [13]:
len(data)

18

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = splitter.split_documents(data)

In [15]:
print("Total number of documents: ",len(docs))

Total number of documents:  96


In [16]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [17]:
from dotenv import load_dotenv
load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [18]:
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [20]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embeddings)

In [23]:
vectorstore

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k":10})

In [32]:
retriever_docs = retriever.invoke("Explain about yolo v8?")
print(retriever_docs[0].page_content)

new multi-scale backbone architecture. IEEE Transac-
tions on Pattern Analysis and Machine Intelligence (TPAMI) ,
43(2):652–662, 2019. 3
[12] Zheng Ge, Songtao Liu, Zeming Li, Osamu Yoshie, and Jian
Sun. OTA: Optimal transport assignment for object detec-
tion. In Proceedings of the IEEE/CVF Conference on Com-
puter Vision and Pattern Recognition (CVPR) , pages 303–
312, 2021. 1
[13] Zheng Ge, Songtao Liu, Feng Wang, Zeming Li, and Jian
Sun. YOLOX: Exceeding YOLO series in 2021. arXiv
preprint arXiv:2107.08430 , 2021. 3[14] Jocher Glenn. YOLOv5 release v7.0. https://github.
com/ultralytics/yolov5/releases/tag/v7.
0, 2022. 3, 7
[15] Jocher Glenn. YOLOv8 release v8.1.0. https :
/ / github . com / ultralytics / ultralytics /
releases/tag/v8.1.0 , 2024. 3, 7
[16] Aidan N Gomez, Mengye Ren, Raquel Urtasun, and Roger B
Grosse. The reversible residual network: Backpropagation
without storing activations. Advances in Neural Information
Processing Systems (NeurIPS) , 2017. 2, 3


In [52]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

In [53]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.5,max_tokens=None,memory=memory)

In [54]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [55]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [56]:
prompt = ChatPromptTemplate([
    ("system",system_prompt),
    ("human","{input}")
])

In [57]:
question_and_answer = create_stuff_documents_chain(llm,prompt)

In [58]:
rag_chain = create_retrieval_chain(retriever,question_and_answer)

In [61]:
response = rag_chain.invoke({"input":"What's new in yolo v1"})
print(response["answer"])

This document discusses YOLOv9 and compares it to other versions like YOLOv7.  It does not contain information about the features introduced in the first YOLO version (v1).

